# **Estruturas criptograficas: TP4 problema 1**

## Dilithium

Este é um algoritmo de assinatura digital pós-quântico que nos permite perceber se aconteceu uma alteração não autorizada, ou seja, o remetente poderá utilizar a assinatura digital para provar, que uma determinada informação não foi modificada e que a mesma veio de um determinado emissor.

### KeyGen

A função keygen é responsável por gerar uma chave pública e uma chave privada (Byte strings), para que possam ser utilizadas pelo emissor e o recetor, para assinar um determinado conteúdo, e verificá-lo respetivamente.

### Sign 

A função sign é capaz de receber uma chave privada sk, e uma mensagem M (byte string), e gerar uma assinatura "sigma". Esta assinatura terá toda a informação necessária para que a função verify possa verificar a validade da mensagem relativamente à sua integridade.


### Verify

A função verify recebe uma chave pública pk, a mensagem que queremos verificar M, e a assinatura sigma, que terá nela toda a informação necessária para extrair os parâmetros para a verificação da mensagem. Após a extração de todos os parâmetros necessários, e consequente verificação, a função retorna um valor booleano True caso a mensagem não tenha sido alterada, e caso contrário retorna False.

In [324]:
from hashlib import shake_256, shake_128
import os
from functools import reduce

In [325]:
class DLTHM:

    def __init__(self, security_strength = 2):
        # ML-DSA-44
        if security_strength == 2:
            self.q = 8380417
            self.d = 13
            self.tau = 39
            self.lam = 128
            self.gama1 = 2^17
            self.gama2 = (self.q-1)/88
            self.k = 4
            self.l = 4
            self.eta = 2
            self.beta = 78
            self.omega = 80
            
        # ML-DSA-65
        elif security_strength == 3:
            self.q = 8380417
            self.d = 13
            self.tau = 49
            self.lam = 192
            self.gama1 = 2^19
            self.gama2 = (self.q-1)/32
            self.k = 6
            self.l = 5
            self.eta = 4
            self.beta = 196
            self.omega = 55
            
        # ML-DSA-87
        elif security_strength == 5:                        
            self.q = 8380417
            self.d = 13
            self.tau = 60
            self.lam = 256
            self.gama1 = 2^19
            self.gama2 = (self.q-1)/32
            self.k = 8
            self.l = 7
            self.eta = 2
            self.beta = 120
            self.omega = 75
        
        self.n = 256
            
        Zq = IntegerModRing(self.q)
        self.Tq = Zq^256

        R.<X> = PolynomialRing(Zq)

        self.Rq = R.quotient(X^256 + 1)
        
    # Função auxiliar para transformar bytes em bits
    def BytesToBits(self, B):
        b = [0] * len(B) * 8 
        B = self.BytesToByteArray(B)
        for i in range(len(B)):
            for j in range(0,8):
                b[8*i+j] = mod(B[i], 2)
                B[i] = B[i] // 2
                
        return b
            
    # Função auxiliar para transformar bits em bytes
    def BitsToBytes(self, b):
        l = len(b) // 8
        B = [0] * l
        for i in range(0,8*l):
            B[i // 8] += ZZ(b[i]) * 2^(mod(i,8))
        return bytes(B)
            
    # Função auxiliar para transformar bytes em bytearray
    def ByteArrayToBytes(self, B):
        return bytes(B)
    
    # Função auxiliar para transformar bytearray em bytes
    def BytesToByteArray(self, Bytes):
        return list(Bytes)
    
    # Função shake_256
    def H(self, bytes, length):
        return shake_256(bytes).digest(length//8)
    
    # Função shake_128
    def H128(self, bytes, length):
        return shake_128(bytes).digest(1024)

    # Função auxiliar para transformar inteiros bits
    def IntegerToBits(self, x, alpha):
        y = []
        for i in range(alpha):
            y.append(ZZ(x) % 2)
            x = ZZ(x) // 2
        return y
    
    def CoefFromThreeBytes(self, b0, b1, b2):
        if b2 > 127:
            b2 -= 128
        z = 2^16 * b2 + 2^8 * b1 + b0
        if z < self.q:
            return z
        else:
            return None
        
    def CoefFromHalfByte(self, b, eta):
        if eta == 2 and b < 15:
            return 2 - (b % 5)
        elif eta == 4 and b < 9:
            return 4 - b
        else:
            return None
    
    def RejNTTPoly(self, rho):
        a = [None]*256
        j = 0
        c = 0
        while j < 256:
            h = self.H128(self.BitsToBytes(rho), 1024)
            a[j] = self.CoefFromThreeBytes(h[c], h[c+1], h[c+2])
            c += 3
            if a[j] is not None:
                j += 1
                
        return a
    
    def RejBoundedPoly(self, rho):
        a = [None]*256
        j = 0
        c = 0
        while j < 256:
            z = self.H(self.BitsToBytes(rho), 2048)[c]
            z0 = self.CoefFromHalfByte(z % 16, self.eta)
            z1 = self.CoefFromHalfByte(z // 16, self.eta)
            if z0 is not None:
                a[j] = z0
                j += 1
            if z1 is not None and j < 256:
                a[j] = z1
                j += 1
            c += 1
        return a

    # Função NTT
    def NTT(self, f):
        f_ = list(f)
        
        k = 1
        len = 128
        while len >= 2:
            start = 0
            while start < 256:
                zeta = mod(17^(self.BitReverse(k)), self.q) 
                k += 1
                for j in range(start, start + len):
                    t = mod(ZZ(zeta) * ZZ(f_[j + len]), self.q)
                    f_[j + len] = mod(ZZ(f_[j]) - ZZ(t), self.q)
                    f_[j] = mod(ZZ(f_[j]) + ZZ(t), self.q)
                    
                
                
                start = start + 2 * len
            len = len // 2
        # f = 8347681
        # for j in range(256):
        #     f_[j] = (f * f_[j]) % self.q
            
        return f_
    
    # Função NTT Inversa
    def NTTInverse(self, f_):
        f = list(f_)
        
        k = 127
        len = 2
        while len <= 128:
            start = 0
            while start < 256:
                zeta = mod(17^(self.BitReverse(k)), self.q)
                k -= 1
                for j in range(start, start + len):
                    t = f[j]
                    f[j] = mod(ZZ(t) + ZZ(f[j + len]), self.q)
                    f[j + len] = mod(ZZ(zeta) * (ZZ(f[j + len]) - ZZ(t)), self.q)
                
                start = start + 2 * len
            len = len * 2
        
        return f
    
    # Função auxiliar para inverter bits de um número com 7 bits
    def BitReverse(self, i):
        return int('{:07b}'.format(i)[::-1], 2)
    
    def ExpandA(self, rho):        
        A = [[None]*self.l for _ in range(self.k)]
        for r in range(self.k):
            for s in range(self.l):
                A[r][s] = self.RejNTTPoly(self.BytesToBits(rho) + self.IntegerToBits(s, 8) + self.IntegerToBits(r, 8))
                
        return A
    
    def ExpandS(self, rho):
        s1 = [None]*self.l
        s2 = [None]*self.k
        for r in range(self.l):
            s1[r] = self.RejBoundedPoly(self.BytesToBits(rho) + self.IntegerToBits(r, 16))
        for r in range(self.k):
            s2[r] = self.RejBoundedPoly(self.BytesToBits(rho) + self.IntegerToBits(r + self.l, 16))
        return s1, s2
      
    def Power2Round(self, r):
        r_plus = mod(r, self.q)

        r0 = self.mod_plus_minus(r_plus, (2**self.d))

        r1 = (ZZ(r_plus) - ZZ(r0)) // (2**self.d)

        return r1, r0  
          
    # Multiplicação de matrizes
    def MatrixMultiplication(self, A, u):
        aux = A.copy()
        res = [0] * self.n
        
        for i in range(self.k):
            aux[i] = self.MultiplyNTTs(A[i], u[i])
            
        for i in range(self.k):    
            res = self.ArrayAddition(res, aux[i])

        return res
    
    # Adição de matrizes
    def MatrixAddition(self, A, B):
        res = []
        for i in range(self.k):
            res.append(self.ArrayAddition(A[i], B[i]))
            
        return res
        
    # Adição de vetores
    def ArrayAddition(self, A, B):
        res = [0] * self.n
        for i in range(self.n):
            res[i] = ZZ(A[i]) + ZZ(B[i])
        
        return res
    
    # Subtração de vetores
    def ArraySubtraction(self, A, B):
        res = [0] * self.n
        for i in range(self.n):
            res[i] = A[i] - B[i]
        
        return res
    
    # Multiplicação de polinómios NTT
    def MultiplyNTTs(self, f, g):
        h = [0] * self.n
        for i in range(128):
            # print(f[2*i])
            # print([2*i + 1])
            # print(g[2*i])
            # print(g[2*i + 1])
            h[2*i], h[2*i + 1] = self.BaseCaseMultiply(f[2*i], f[2*i + 1], g[2*i], g[2*i + 1], 17^(2* self.BitReverse(i) + 1))
        return h
    
    
    def BaseCaseMultiply(self, a0, a1, b0, b1, y):
        c0 = mod((a0 * b0) + (a1 * b1 * y), self.q)
        c1 = mod((a0 * b1) + (a1 * b0), self.q)
        return c0, c1 
    
    def round(self, x):
        return int(x + 0.5)
    
    def bitlen(self, a):
        return len(bin(a)) - 2
    
    def mod_plus_minus(self, x, y):
        result = (ZZ(x + y // 2) % y) - (y // 2)
        return result
    
    def SimpleBitPack(self, w, b):
        z = []
        for i in range(256):
            z += self.IntegerToBits(w[i], self.bitlen(b))
            
        return self.BitsToBytes(z)
    
    def SimpleBitUnpack(self, v, b):
        c = self.bitlen(b)
        z = v
        w = [0] * 256
        for i in range(256):
            w[i] = self.BitsToInteger(z[i*c:(i+1)*c])   
            
        return w
    
    def BitsToInteger(self, y):
        x = 0
        for i in range(len(y)):
            x = 2*x + y[len(y) - i - 1]
            
        return x
    
    def BitPack(self, w, a, b):
        z = []
        for i in range(256):
            z += self.IntegerToBits(b - w[i], self.bitlen(a + b))
            
        return self.BitsToBytes(z)
    
    def BitUnpack(self, v, a, b):
        c = self.bitlen(a + b)
        z = self.BytesToBits(v)
        
        w = [0] * 256
        for i in range(256):
            w[i] = b - self.BitsToInteger(z[i*c:(i+1)*c])
        
        return w
    
    # Codifica a public key
    def pkEncode(self, rho, t1):

        pk = rho

        for i in range(self.k):
            pk += self.SimpleBitPack(t1[i], 2 ** (self.bitlen(self.q - 1) - self.d) - 1)

        return pk
    
    # Descodifica a public key
    def pkDecode(self, pk):
        rho = pk[:32]
        z = pk[32:]
        t1 = []
        for i in range(self.k):
            t1.append(self.SimpleBitUnpack(z[i * 320: (i + 1) * 320], 2**(self.bitlen(self.q - 1)-self.d) - 1))
        
        return rho, t1
    
    # Codifica a secret key
    def skEncode(self, rho, K, tr, s1, s2, t0):
        sk = rho

        sk += K

        sk += tr

        for i in range(self.l):
            sk += self.BitPack(s1[i], self.eta, self.eta)

        for i in range(self.k):
            sk += self.BitPack(s2[i], self.eta, self.eta)

        for i in range(self.k):
            sk += self.BitPack(t0[i], 2**(self.d - 1) - 1, 2**(self.d - 1))

        return sk
    
    # Descodifica a secret key
    def skDecode(self, sk):
        rho = sk[:32]
        K = sk[32:64]
        tr = sk[64:128]
        
        v1 = 128 +((32 * self.bitlen(2 * self.eta)) * self.l)
        y = sk[128:v1]
        v2 = v1 + ((32 * self.bitlen(2 * self.eta)) * self.k)
        z = sk[v1:v2]
        w = sk[v2:]
        
        
        s1 = [None]*self.l
        for i in range(self.l):
            s1[i] = self.BitUnpack(y[i * 96: (i + 1) * 96], self.eta, self.eta)
                
        s2 = [None]*self.k
        for i in range(self.k):
            s2[i] = self.BitUnpack(z[i * 96: (i + 1) * 96], self.eta, self.eta)
            
        t0 = [[0] * self.n for _ in range(self.k)]
        for i in range(self.k):
            t0[i] = self.BitUnpack(w[i * 416: (i + 1) * 416], 2**(self.d - 1) - 1, 2**(self.d - 1))
        
        return rho, K, tr, s1, s2, t0
    
    def ExpandMask(self, rho, mu):
        c = 1 + self.bitlen(self.gama1 - 1)
        
        s = []

        for r in range(self.l):
            n = self.IntegerToBits(mu + r, 16)
            n_bytes = self.BitsToBytes(n)  # Convert bits to bytes if needed
            v = []
            
            for i in range(32 * c):
                hash_input = rho + n_bytes
                hash_output = self.H(hash_input, 1024)
                v.append(hash_output[i % len(hash_output)])  # Collect necessary hash output bytes
            
            s_r = self.BitUnpack(v, self.gama1 - 1, self.gama1)
            s.append(s_r)

        return s
    
    def Decompose(self, r):
        r_plus = mod(r, self.q)
        r0 = mod(r_plus, 2*self.gama2)
        
        if ZZ(r_plus) - ZZ(r0) == self.q - 1:
            r1 = 0
            r0 = ZZ(r0) - 1
        else:
            r1 = (ZZ(r_plus) - ZZ(r0)) // 2*self.gama2
            
        return (r1, r0)
    
    def HighBits(self, r):
        (r1, r0) = self.Decompose(r)
        return r1
    
    def LowBits(self, r):
        (r1, r0) = self.Decompose(r)
        return r0
    
    # Função para gerar as chaves
    def keygen(self):
        
        zeta = os.urandom(32)
        
        temp_bytes = self.H(zeta, 1024)
        # temp_bits = self.BytesToBits(temp_bytes)
        
        rho, rho_, K = temp_bytes[:32], temp_bytes[32:96], temp_bytes[96:]
        
        A_hat = self.ExpandA(rho)
        # print(A_hat)
        
        s1, s2 = self.ExpandS(rho_)

        ntt_s1 = [] 
        for i in range(self.l):
            ntt_s1.append(self.NTT(s1[i]))
        
        t = [
		reduce(self.ArrayAddition, [
                self.MultiplyNTTs(A_hat[i][j], ntt_s1[j])
                for j in range(self.l)
            ] + [s2[i]])
            for i in range(self.k)
        ]
        
        t1 = [[0] * self.n for _ in range(self.k)]
        t0 = [[0] * self.n for _ in range(self.k)]

        for i in range(self.k):
            for j in range(self.n):
                t1[i][j], t0[i][j] = self.Power2Round(t[i][j])
        
        pk = self.pkEncode(rho, t1)
        
        tr =  self.H(pk, 512)
        
        sk = self.skEncode(rho, K, tr, s1, s2, t0)
        
        
        return pk, sk
    
    def w1Encode(self, w1):
        w1_hat = []
        
        for i in range(self.k):
            w1_hat += self.BytesToBits(self.SimpleBitPack(w1[i], (self.q - 1) / (2 * self.gama2) - 1))
        
        return w1_hat
    
    def InfinityNorm(self, w, num):
        for i in range(len(w)):
            for j in range(self.n):
                if abs(ZZ(w[i][j])) >= num:
                    return False
        return True
    
    def MakeHint(self, z, r):
        r1 = self.HighBits(r)
        v1 = self.HighBits(r + z)
        
        if r1 != v1:
            return 0
        else:
            return 1

    def SampleInBall(self, rho):
        c = [0] * 256
        
        k = 8
        
        for i in range(256 - self.tau, 256):
            while self.H(self.BitsToBytes(rho), 1024)[k] > i:
                k += 1
            
            j = self.H(self.BitsToBytes(rho), 1024)[k]
            
            ci = c[j]
            c[j] = (-1) ** self.H(self.BitsToBytes(rho), 1024)[i + self.tau - 256]
            c[i] = ci
            
            k += 1
        
        return c
    
    # Codifica a assinatura
    def sigEncode(self, c_, z, h):
        sig = c_
        
        for i in range(self.l):
            sig += self.BitPack(z[i], self.gama1 - 1, self.gama1)
        

        sig += bytes(self.HintBitPack(h))
        return sig
        
    
    def HintBitPack(self, h):
        y = [0] * (self.omega + self.k)
        index = 0
        for i in range(self.k):
            for j in range(self.n):
                if h[i][j] != 0:
                    y[index] = j
                    index += 1
            y[self.omega + i] = index    
                
        return y
    
    # Função para assinar
    def sign(self, sk, m):
        
        rho, K, tr, s1, s2, t0 = self.skDecode(sk)
        
        s1_hat = [self.NTT(s1[i]) for i in range(self.l)]
        s2_hat = [self.NTT(s2[i]) for i in range(self.k)]
        t0_hat = [self.NTT(t0[i]) for i in range(self.k)]
        
        A_hat = self.ExpandA(rho)
        
        mu = self.H(tr + m, 512)
        
        rnd = os.urandom(32)
        
        rho_ = self.H(K + rnd + mu, 512)
        
        k = 0
        (z, h) = (None, None)
        
        while (z, h) == (None, None):
            y = self.ExpandMask(rho_, k)
            # print(y)
            # print(len(y))
            y_ntt = [self.NTT(y[i]) for i in range(self.l)]
            
            w = [self.NTTInverse(reduce(self.ArrayAddition, [
                    self.MultiplyNTTs(A_hat[i][j], y_ntt[j])
                    for j in range(self.l)
                ]))
                for i in range(self.k)
            ]
            
            w1 = [[0] * self.n for _ in range(self.k)]

            for i in range(self.k):
                for j in range(self.n):
                    w1[i][j] = self.HighBits(w[i][j])
                        
            c_ = self.H(mu + self.BitsToBytes(self.w1Encode(w1)), 2 * self.lam)
            # print("c_", c_)
            c1_ = c_[:32]
            c2_ = c_[32:]
            
            c = self.SampleInBall(self.BytesToBits(c1_))
            
            c_hat = self.NTT(c)
            cS1 = [self.NTTInverse(reduce(self.ArrayAddition, 
                [self.MultiplyNTTs(c_hat, s1_hat[j])])
                ) for j in range(self.l)
            ]
            
            cS2 = [self.NTTInverse(reduce(self.ArrayAddition, 
                [self.MultiplyNTTs(c_hat, s2_hat[j])])
                ) for j in range(self.k)
            ]

            z = [self.ArrayAddition(y[i], cS1[i]) for i in range(self.k)]
            tt = [self.ArraySubtraction(w[i], cS2[i]) for i in range(self.k)]
            r0 = [[0] * self.n for _ in range(self.k)]
            for i in range(self.k):
                for j in range(self.n):
                    r0[i][j] = self.LowBits(tt[i][j])
            
            if self.InfinityNorm(z, self.gama1 - self.beta) or self.InfinityNorm(r0, self.gama2 - self.beta):
                (z, h) = (None, None)
            else:
                cT0 = [self.NTTInverse(reduce(self.ArrayAddition, 
                    [self.MultiplyNTTs(c_hat, t0_hat[j])])
                    ) for j in range(self.k)
                ]
                
                sub = [self.ArrayAddition(w[i], cS2[i]) for i in range(self.k)]
                su = [self.ArrayAddition(sub[i], cT0[i]) for i in range(self.k)]
                
                h = [[0] * len(cT0[0]) for _ in range(len(cT0))]
                for i in range(len(cT0)):
                    for i in range(len(cT0)):
                        h[i][j] = self.MakeHint(-cT0[i][j], su[i][j])
                                    
                count = 0
                for i in range(len(h)):
                    if h[i] == 1:
                        count += 1

                if self.InfinityNorm(cT0, self.gama2) or count > self.omega:
                    (z, h) = (None, None)
                    
            k += self.l
        
        for i in range(self.l):
            for j in range(self.n):
                z[i][j] = self.mod_plus_minus(z[i][j], self.q)

        sig = self.sigEncode(c_, z, h)
        
        
        return sig

        
    def Verify(self, pk, M, sig):
        
        rho, t1 = self.pkDecode(pk)

        
        

### Geração de chaves

In [326]:
dilithium = DLTHM(2)
print("KEYGEN")
pk, sk = dilithium.keygen()

print("Public Key: ", pk)
print("Secret Key: ", sk)

KEYGEN
Public Key:  b'\x15\x861\x84\xfcs\x83\xe5\x8f\xd7!/\xb3LE\xc5\xa2l%\xb6\xeb\xe9\xe7A\x8ar\x90\x12\x807\xea\xec8\xc5T\xce}v;z\xe2x#\xe4\x85\xd9E\xb9\xff\r`\xfa\xb2#\xe0\x805\x06\xa6\x89X\xf4)\x18=\x87\xfb3\x84.f;\xeb Oe\xe6\x83<\xf6\xe1\xf2\\\x0c\xabXlB\x0c\xcf\x87\xa5\xe8\xcdR\xed\xaa\xe8^o\x0b\x85+\x9fV)x\xb8\x1fY\xda(Y)t\xd8\x80\xbe\xfd\xd3\xefL.\xabxT\xcb+\xd8e2\x83>\x0c\x99B \xdfi\'^B\xe3\x9e\xcb\xcb,6(\xb6\xcb6\xe3\xdd]}+\x86\x14\x9f\x0521\x8c\xcf(\xaep\xee\xce`\x86\x00I\xc7\x87\xc9\xd5`\xbe\x881\xf7\x95w\xbe\xc7Kk@\xec\x0c\x05\xc0\x8c\xe0\x8f\xf6\xd6>\xff+S#\x03\xaeT\x89\xcb\x9d\xe9\x02B\x80s\xd3${a\xbc\xfaR\xeax\xab\xbf]\x8f\x1f\xd0\xc2\x15\xc2lLdE\n\xea\n\x0b\x0f\xa1cBNH\xa7\x08^\xb2\xad\xd0\xb10\xce\x97\xf3\x9a\xdc)\x14C\xffb\xa8\x9c\xbfi\x9f\xdb}\xf6\x9b\x8e\xf7\xc4\xe0\x08\x15J\xb2Y>op\x05\x84Cr]\xbf_l9\xcf\xad\x9d\x8fF\xd0\x93`g\x97\xecW\xbc\xd6z\x1eZ\xdbww\xa3g\xb9s\xed\xe2s\xdd\xb6&AaZ\xe8\x12J\x99#\xd3\x18\xc1\xfb\xd2\x9au\xc3KG\xceN\x1f\xee]\xf8\xef\xdb\x1a\xa9/\

### Assinatura

In [327]:
print("SIGN")
sig = dilithium.sign(sk, b"Hello World")

print("Assinaura (sigma): ", sig)

SIGN
Assinaura (sigma):  b'\x15\x8bg\xb8\xf4\xe45Va\xdd\x87\xecY\x1f&q\x07\xf4\xf3JP\x0eJN%\xe6u\xe05\xaaR\x82\xf5\xa1\x8a\xf5\x15\x89\xdfw\xf2\x1d\xed|\xfaC\x80N\xfc\xae1\x19e\xd2#\xb4\xadx\xb9y\xef\x90\x86\x90k\x838\xeceS\x1a\x93\xd0N\xfd\x86\xde\xf1s\x94j{G\xe2~\xc9\xe6_\xcc\x12tv\x96\xe5\xc1\xa3\xe0\x14\xd7\x89\x1a\xbc\xacz\xd9\x0c\xe8.!\xe7\x8b\xbb+\xce\xba\x9e\xa3Z0\xc4/\xab`;\xb3\xa9-\xc2\xd4\t\t\xabL\x1d\x9f\x83u\xbfuJ\xd3H\n\x08\x06\xf5\x8f\xe0\xf1\x07<i\xb8Q\x16\xd1A<\xc8\x99\x13\x9c\x12\xd8\xf9\x0bn\xb5\x04\xd6\x84Kw\xa4\x7f\xf1g_\xa1\x0f\x95Mt\x0b63I\xc8\xb8\x15\x12\x95\xf2a\x12\x05\xecy\x81"\r\xae\xda\r|=\x01)W\x10\xc2\xd8\xec\x97cn\x8d\xf3P\x0b\xd1G\x97(#\xa3\xb5T\xb9\x82`\xe8O\xea\x7fX\tr<\x07\xa16\x1d\x88!\xfc+\xf5\x82\xd6\x00\xe8\xda\x08\x89b\xc0\x19\xf1\x1f\x19\xd8\xd1\x0e\x1f\x98\xa7b\xa1[\xd7\xfd\xe1\x1a\x97oOs*@Ca\xf2\xdb\xd5\xc1\xcc%&\xacjB)Mf\xcem\xfa\xb1\x02\x94\x92+)\xdc\xfau\xe9}\xb3\xf0\xf2\xc5\xff\xd8\x98_\xb2I\x07\r\xae\xf9)@\x84\xf4\xc1\x8aU\xceUV\x1a\xffZ

### Verificação

In [328]:
print("VERIFY")
# dilithium.Verify(pk, b"Hello World", sig)

VERIFY
